In [1]:
import math
import datetime
import numpy as np
import pickle

import methodsMLinterns
from customAutoencoder import ModelType, AutoencoderEnsemble, ExperimentPerformance

Using TensorFlow backend.


In [2]:
stocks = ['DNB', 'NRG', 'CL', 'ANTM', 'NEE', 'PAYX', 'VAR', 'NI', 'MNST', 'JNJ', 'TGNA', 'NOV', 'FIS', 'BLK', 'HBI', 'NVDA', 'DLTR', 'MRO', 'EMN', 'AMT', 'FLR', 'IBM', 'BK', 'NFX', 'AGN', 'LRCX', 'DIS', 'LH', 'C', 'MNK']

In [3]:
with open("pickles/CVhyperasSearchBestParameters_z_30.p",'rb') as f:
    bparams = pickle.load(f)
with open("pickles/CVhyperasSearchParameters_z_30.p",'rb') as f:
    params, _, _ = pickle.load(f)
bparams["l2"]=0.005
bparams["autoencoderLoss"]='cosine_proximity'
bparams["batch_size"]=256
print(params[0])
for i in range(0,len(params)):
    if params[i] == bparams:
        print(i)
print(bparams)

{'autoencoderLoss': 'mean_squared_error', 'batch_size': 256, 'd': 0.5540312274525752, 'd_1': 0.1702403465020466, 'l2': 0.08}
24
{'l2': 0.005, 'batch_size': 256, 'd': 0.0068486252678771065, 'd_1': 0.06989977789379659, 'autoencoderLoss': 'cosine_proximity'}


In [4]:
import glob
import os
import datetime

filenames = []
unique = []

i=0
temp=[]
for file in glob.glob("weightsCV/test_*z_30*.hdf5"):
    if i==6:
        i=0
        filenames.append(temp)
        temp=[]
    name = file[15:34]
    temp.append(file[10:-5])
    if name not in unique:
        unique.append(name)    
    i+=1
filenames.append(temp)

print(len(filenames))
score = 0
for i, u in enumerate(unique):
    for f in filenames[i]:
        score += u in f
print(score)
print(filenames[0])

with open("pickles/CV_setsIndexes.p",'rb') as f:
    sets = pickle.load(f)

100
600
['test_2017-06-19_10:46:57_z_30_f0_0', 'test_2017-06-19_10:46:57_z_30_f0_1', 'test_2017-06-19_10:46:57_z_30_f0_2', 'test_2017-06-19_10:46:57_z_30_f1_0', 'test_2017-06-19_10:46:57_z_30_f1_1', 'test_2017-06-19_10:46:57_z_30_f1_2']


In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [6]:
features_1p4_extra =['aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'abj', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acr', 'acw', 'acx', 'acy', 'adi', 'adj', 'adl', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aex', 'aey', 'aez', 'afa', 'afj', 'afl', 'afo', 'afp', 'afq', 'afr', 'afs', 'aft', 'afu', 'afv', 'afw', 'afx', 'afy', 'afz', 'aga', 'agb', 'agc', 'agd', 'age', 'agf', 'agg', 'agh', 'agi', 'agj', 'agk', 'agl', 'agm', 'agn', 'ago', 'agp', 'agq', 'agr', 'ags', 'agt', 'agu', 'agv', 'agw', 'agx', 'agy', 'ahf', 'ahg', 'ahh', 'ahi', 'ahj', 'ahk', 'ahl', 'ahm', 'ahn', 'aho', 'zhq', 'zhr', 'zhs', 'zht', 'zhu', 'zhv', 'zhw', 'ziy', 'zjb', 'zjc', 'zjd', 'zje', 'zjf', 'zjg', 'zjh', 'zji', 'zjj', 'zjk', 'zjl', 'zjm', 'zjn', 'zjo', 'zjp', 'zjq', 'zjr', 'zjs', 'zjt', 'zju', 'zkg', 'zkl', 'zkm', 'zkn', 'zkx', 'zky', 'zla', 'zld', 'zle', 'zlf', 'zlg', 'zlh', 'zli', 'zlj', 'zlk', 'zll', 'zlm', 'zln', 'zlo', 'zlp', 'zlq', 'zlr', 'zls', 'zlt', 'zlu', 'zlv', 'zlw', 'zlx', 'zly', 'zlz', 'zma', 'zmb', 'zmc', 'zmd', 'zme', 'zmf', 'zmg', 'zmh', 'zmm', 'zmn', 'zmo', 'zmp', 'zmy', 'zna', 'znd', 'zne', 'znf', 'zng', 'znh', 'zni', 'znj', 'znk', 'znl', 'znm', 'znn', 'zno', 'znp', 'znq', 'znr', 'zns', 'znt', 'znu', 'znv', 'znw', 'znx', 'zny', 'znz', 'zoa', 'zob', 'zoc', 'zod', 'zoe', 'zof', 'zog', 'zoh', 'zoi', 'zoj', 'zok', 'zol', 'zom', 'zon', 'zou', 'zov', 'zow', 'zox', 'zoy', 'zoz', 'zpa', 'zpb', 'zpc', 'zpd', 'zpe']
features_1p4_extra_a=[f for f in features_1p4_extra if f[0]=='a']
features_1p4_extra_z=[f for f in features_1p4_extra if f[0]=='z']
assert(len(features_1p4_extra_a)+len(features_1p4_extra_z)==len(features_1p4_extra))

In [7]:
date_test_set = datetime.date(2016, 1, 1)

clf_portfolio_dic = methodsMLinterns.ClassificationPortfolio(stocks=stocks, minutes_forward=30)
clf_portfolio_dic.loadData()
clf_portfolio_dic.cleanUpData(features_1p4_extra)
clf_portfolio_dic.getTrainTestSetDate(date_test_set)

### Test on stocks separately

In [8]:
def testOnSeparateStocks(autoList, feat, votingType='soft'):
    n = len(autoList)
    trainNumOfTrades = np.zeros((clf_portfolio_dic.N_stocks,3))
    valNumOfTrades = np.zeros((clf_portfolio_dic.N_stocks,3))
    testNumOfTrades = np.zeros((clf_portfolio_dic.N_stocks,))
    acc_train_autoencoder = np.zeros((n,clf_portfolio_dic.N_stocks,3))
    acc_val_autoencoder = np.zeros((n,clf_portfolio_dic.N_stocks,3))
    acc_test_autoencoder = np.zeros((n,clf_portfolio_dic.N_stocks,))

    # difficulty is to recreate the validation set for each stock
    indexCount = np.zeros((3,))
    for k, stock in enumerate(clf_portfolio_dic.stocks):
        name = "%s30"%stock
        print(k,name)
        
        # extract stock data
        X_train = np.array(clf_portfolio_dic.X_train_dic[name][feat])
        x_test = np.array(clf_portfolio_dic.X_test_dic[name][feat])
        Y_train = (clf_portfolio_dic.y_train_dic[name]+1)/2
        y_test = (clf_portfolio_dic.y_test_dic[name]+1)/2
        
        # transform ys to one-hot vectors
        y_t = np.zeros((Y_train.shape[0], 2))
        y_t[np.arange(Y_train.shape[0]), Y_train.astype('int32')] = 1
        Y_train = y_t

        y_t = np.zeros((y_test.shape[0], 2))
        y_t[np.arange(y_test.shape[0]), y_test.astype('int32')] = 1
        y_test = y_t
        
        n = Y_train.shape[0]
        for j in range(3):
            print("Fold ",j)
            # split train into val+train
            train_idx = sets[j][0]
            val_idx = sets[j][1]
            train_mask = np.array([(i+indexCount[j] in train_idx) for i in range(n)])
            val_mask = np.array([(i+indexCount[j] in val_idx) for i in range(n)])

            x_train, y_train, x_val, y_val = X_train[train_mask,:], Y_train[train_mask,:], X_train[val_mask,:], Y_train[val_mask,:]
            indexCount[j] += n
            assert(n==y_train.shape[0]+y_val.shape[0])
            
            # store numbers of trades
            trainNumOfTrades[k,j] = y_train.shape[0]
            valNumOfTrades[k,j] = y_val.shape[0]
            testNumOfTrades[k] = y_test.shape[0]
    
            # compute accuracies
            for i, a in enumerate(autoList):
                print(i)
                hardVoting_train, softVoting_train, productVoting_train, _ = a.predict(x_train)
                hardVoting_val, softVoting_val, productVoting_val, _ = a.predict(x_val)
                hardVoting_test, softVoting_test, productVoting_test, _ = a.predict(x_test)

                if votingType == 'soft':
                    y_pred_train = softVoting_train
                    y_pred_val = softVoting_val
                    y_pred_test = softVoting_test
                elif votingType == 'hard':
                    y_pred_train = hardVoting_train
                    y_pred_val = hardVoting_val
                    y_pred_test = hardVoting_test
                elif votingType == 'product':
                    y_pred_train = productVoting_train
                    y_pred_val = productVoting_val
                    y_pred_test = productVoting_test

                acc_train = (100 * (np.argmax(y_pred_train,1) == np.argmax(y_train,1))).mean()
                acc_val = (100 * (np.argmax(y_pred_val,1) == np.argmax(y_val,1))).mean()
                acc_test = (100 * (np.argmax(y_pred_test,1) == np.argmax(y_test,1))).mean()
                acc_train_autoencoder[i,k,j] = acc_train
                acc_val_autoencoder[i,k,j] = acc_val
                acc_test_autoencoder[i,k] = acc_test
                    
    return(acc_train_autoencoder, acc_val_autoencoder, acc_test_autoencoder,
           trainNumOfTrades, valNumOfTrades, testNumOfTrades)


In [9]:
ensList = []
for n in filenames[:10]:
    print(n)
    ensemble = AutoencoderEnsemble(n)
    ensemble.loadModels(ModelType.Merged, "weightsCV")
    ensList.append(ensemble)



['test_2017-06-19_10:46:57_z_30_f0_0', 'test_2017-06-19_10:46:57_z_30_f0_1', 'test_2017-06-19_10:46:57_z_30_f0_2', 'test_2017-06-19_10:46:57_z_30_f1_0', 'test_2017-06-19_10:46:57_z_30_f1_1', 'test_2017-06-19_10:46:57_z_30_f1_2']
weightsCV/test_2017-06-19_10:46:57_z_30_f0_0.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 120)           0           input_1[0][0]                    
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           12100       dropout_1[0][0]                  
______________

weightsCV/test_2017-06-19_10:46:57_z_30_f1_1.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_13 (Dropout)             (None, 120)           0           input_5[0][0]                    
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 100)           12100       dropout_13[0][0]                 
____________________________________________________________________________________________________
dropout_14 (Dropout)             (None, 100)           0           dense_9[0][0]                    
_________________________________________

weightsCV/test_2017-06-19_10:56:19_z_30_f0_2.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_25 (Dropout)             (None, 120)           0           input_9[0][0]                    
____________________________________________________________________________________________________
dense_17 (Dense)                 (None, 100)           12100       dropout_25[0][0]                 
____________________________________________________________________________________________________
dropout_26 (Dropout)             (None, 100)           0           dense_17[0][0]                   
_________________________________________

['test_2017-06-19_11:06:07_z_30_f0_0', 'test_2017-06-19_11:06:07_z_30_f0_1', 'test_2017-06-19_11:06:07_z_30_f0_2', 'test_2017-06-19_11:06:07_z_30_f1_0', 'test_2017-06-19_11:06:07_z_30_f1_1', 'test_2017-06-19_11:06:07_z_30_f1_2']
weightsCV/test_2017-06-19_11:06:07_z_30_f0_0.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_13 (InputLayer)            (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_37 (Dropout)             (None, 120)           0           input_13[0][0]                   
____________________________________________________________________________________________________
dense_25 (Dense)                 (None, 100)           12100       dropout_37[0][0]                 
______________

weightsCV/test_2017-06-19_11:06:07_z_30_f1_1.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_17 (InputLayer)            (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_49 (Dropout)             (None, 120)           0           input_17[0][0]                   
____________________________________________________________________________________________________
dense_33 (Dense)                 (None, 100)           12100       dropout_49[0][0]                 
____________________________________________________________________________________________________
dropout_50 (Dropout)             (None, 100)           0           dense_33[0][0]                   
_________________________________________

...

weightsCV/test_2017-06-19_11:53:28_z_30_f1_1.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_41 (InputLayer)            (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_121 (Dropout)            (None, 120)           0           input_41[0][0]                   
____________________________________________________________________________________________________
dense_81 (Dense)                 (None, 100)           12100       dropout_121[0][0]                
____________________________________________________________________________________________________
dropout_122 (Dropout)            (None, 100)           0           dense_81[0][0]                   
_________________________________________

weightsCV/test_2017-06-19_12:08:51_z_30_f0_2.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_45 (InputLayer)            (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_133 (Dropout)            (None, 120)           0           input_45[0][0]                   
____________________________________________________________________________________________________
dense_89 (Dense)                 (None, 100)           12100       dropout_133[0][0]                
____________________________________________________________________________________________________
dropout_134 (Dropout)            (None, 100)           0           dense_89[0][0]                   
_________________________________________

['test_2017-06-19_12:15:45_z_30_f0_0', 'test_2017-06-19_12:15:45_z_30_f0_1', 'test_2017-06-19_12:15:45_z_30_f0_2', 'test_2017-06-19_12:15:45_z_30_f1_0', 'test_2017-06-19_12:15:45_z_30_f1_1', 'test_2017-06-19_12:15:45_z_30_f1_2']
weightsCV/test_2017-06-19_12:15:45_z_30_f0_0.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_49 (InputLayer)            (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_145 (Dropout)            (None, 120)           0           input_49[0][0]                   
____________________________________________________________________________________________________
dense_97 (Dense)                 (None, 100)           12100       dropout_145[0][0]                
______________

weightsCV/test_2017-06-19_12:15:45_z_30_f1_1.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_53 (InputLayer)            (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_157 (Dropout)            (None, 120)           0           input_53[0][0]                   
____________________________________________________________________________________________________
dense_105 (Dense)                (None, 100)           12100       dropout_157[0][0]                
____________________________________________________________________________________________________
dropout_158 (Dropout)            (None, 100)           0           dense_105[0][0]                  
_________________________________________

weightsCV/test_2017-06-19_12:22:36_z_30_f0_2.hdf5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_57 (InputLayer)            (None, 120)           0                                            
____________________________________________________________________________________________________
dropout_169 (Dropout)            (None, 120)           0           input_57[0][0]                   
____________________________________________________________________________________________________
dense_113 (Dense)                (None, 100)           12100       dropout_169[0][0]                
____________________________________________________________________________________________________
dropout_170 (Dropout)            (None, 100)           0           dense_113[0][0]                  
_________________________________________

In [10]:
with open("_pickles/ensList.p",'wb') as f:
    pickle.dump( ensList, f, protocol=pickle.HIGHEST_PROTOCOL)

PicklingError: Can't pickle <class 'module'>: attribute lookup module on builtins failed

In [11]:
features = features_1p4_extra_z
expList = []
for votingType in ['soft']:#,'hard']:
    name = "ensemble best %s voting"%votingType
    print(name)
    e = ExperimentPerformance(
        methodName=name, 
        stocks=stocks, originalFeatures=features)
    expList.append(e)
                        
    acc_train_vec, acc_val_vec, acc_test_vec, trainNumOfTrades, valNumOfTrades, testNumOfTrades = testOnSeparateStocks(ensList,features,votingType)    
    

ensemble best soft voting
0 DNB30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
1 NRG30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
2 CL30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
3 ANTM30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
4 NEE30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
5 PAYX30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
6 VAR30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
7 NI30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
8 MNST30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
9 JNJ30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3
4
5
6
7
8
9
Fold  2
0
1
2
3
4
5
6
7
8
9
10 TGNA30
Fold  0
0
1
2
3
4
5
6
7
8
9
Fold  1
0
1
2
3

In [12]:
e = expList[0]
print(acc_train_vec[0,:].shape,trainNumOfTrades.shape)

(30, 3) (30, 3)


In [13]:
for i, e in enumerate(expList):
    e.setTrainResults(acc_train_vec[i,:], trainNumOfTrades)
    e.setValResults(acc_val_vec[i,:], valNumOfTrades)
    e.setTestResults(acc_test_vec[i,:], testNumOfTrades)
    print("")
    print("%s | %s"%(e.getAccuracy(set='train',avg=True),e.getAccuracyMinusSigma(set='train',avg=True)))
    print("%s | %s"%(e.getAccuracy(set='val',avg=True),e.getAccuracyMinusSigma(set='val',avg=True)))
    print("%s | %s"%(e.getAccuracy(set='test'),e.getAccuracyMinusSigma(set='test')))


52.4464345429 | 51.2818453911
52.4464345429 | 51.0109295802
51.0779436153 | 49.6729528737


In [ ]:
print(len(expList))

In [87]:
with open("pickles/acc_autoencoder_logit_ensemble.p",'wb') as f:
    pickle.dump( expList, f, protocol=pickle.HIGHEST_PROTOCOL)